# Target Propagation - Tutorial 1
## Standard Target Propagation

A simple implementation of target propagation to confirm that it works.

In [ ]:
import initialize

In [ ]:
from torchvision.datasets import FashionMNIST, CIFAR10
from torchvision import transforms

from tools.modules import Sign, Sampler, Clamp
from tools.training import train, classify
from tools import training
from tools.learners.target_propv2 import DeepLinearTPLearner
from tools.learners.target_propv2 import BaselineLearner1
from functools import partial
from torch import nn

import numpy as np

from tools.training import train, classify
import zenkai
from functools import partial

from tools.learners.target_propv2 import BaselineLearner1, select_act, LinearTPLearner, TPAltStepTheta, DiffTPStepX, TPStepTheta, TPStepX, DeepLinearTPLearner

# Steps

1) Create each layer (AutoencoderLearner)
2) Create the TargetPropLearner
3) Run the training on the baseline
4) Run the training on the target propagation learner 

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_dataset = CIFAR10(
    '../../Datasets/',
    transform=transform, download=True
)

testing_dataset = CIFAR10(
    '../../Datasets/', train=False,
    transform=transform, download=True
)
import math
k = math.prod(testing_dataset[0][0].shape)

losses = {}
classifications = {}


epoch_results = {}


In [ ]:
def get_key(predictor: str, activation: str) -> str:
    return f'{predictor}_{activation}'



# Baseline Learner

Train baseline learners on 
 - LeakyReLU 
 - Sign
 - Stochastic

Use straight-through-estimators for the latter two

In [ ]:
act = 'leaky_relu'
activation = nn.LeakyReLU

learner = BaselineLearner1(
    k, 300, 300, 300, 10, activation=activation
)

key = 'Baseline - LeakyReLU'
losses[key], epoch_results[key] = train(learner, training_dataset, 40, device='cpu')
classifications[key] = classify(learner, testing_dataset)

In [ ]:
# import numpy as np


# print(np.mean(epoch_results['loss']))

# Fashion MNIST 0.39
# CIFAR 1.20

In [ ]:
# import pickle

# losses = {'baseline': baseline_loss}

# with open('results/t2x1_loss_cifar_results2.pkl', 'wb') as file:
#     pickle.dump(losses, file)

# TargetPropLearner

Train target propagation learners using "LeakyReLU", "Sign", and "Stochastic" activation functions.

In [ ]:
from torch import nn


append = '3layer'

# for act in ['leaky_relu', 'sign', 'stochastic']:
for act, rec_weight in [
    # ('leaky_relu', None),
    # ('binary', None),
    # 'sampler', None),
    # ('clamp', None),
    ('sign', None),
    # ('sigmoid', None)
    # ('leaky_relu', 0.1),
    # ('sign', 0.1),
    # ('stochastic', 1.0),
    # ('stochastic', None),
]:
    print('Activation: ', act)

    key, act, in_act = select_act(act, rec_weight)

    key += f'_{append}'
    step_x = TPStepX.factory()

    step_theta = TPAltStepTheta.factory(
        zenkai.NNLoss('MSELoss', 'mean'),
        zenkai.NNLoss('MSELoss', 'mean'),
        1e-4, 1e-4, True, True, rec_weight, 1.0, # reg=1e0
    )

    i = 0
    out_x_lr = 1.0
    print(k)
    learner = DeepLinearTPLearner(
        k, 200, 200, 200, 10, step_theta, step_x, out_x_lr,
        act, act, in_act, True, False, 1e-4, dropout_p=None, 
        gaussian_noise=0.1, share_weights=False
    )

    losses[key], epoch_results[key] = train(
        learner, training_dataset, 40, 
        device='cpu', callback=None, batch_size=128
    )
    classifications[key] = classify(learner, testing_dataset)

In [ ]:
np.mean(epoch_results['leaky_relu']['loss'])

In [ ]:

keys = list(losses.keys())
values = list(losses.values())

training.plot_loss_line(
    values, keys, 
    'Training Loss', save_file='images/t2x1_target_prop_2024_10_3_1.png'
)

In [ ]:
classification = '\n'.join(f'{k}: {v}' for k, v in classifications.items())
print(classification)

In [ ]:
import pickle

with open('results/t2x1_loss_results1.pkl', 'wb') as file:
    pickle.dump(losses, file)